# License Management

In this demo we will show how to manage user licenses on ArcGIS Online.

### Setup the Environment

In [2]:
from IPython import display
import pandas as pd
from arcgis.gis import GIS

In [3]:
gis = GIS("home")

/opt/conda/lib/python3.9/site-packages/arcgis/gis/__init__.py:597: UserWarning: You are logged on as naubry_geosaurus with an administrator role, proceed with caution.
  warnings.warn(


### Access the License Manager

In [3]:
lm = gis.admin.license

In [5]:
licenses = lm.all()
licenses

[<ArcGIS Survey123 License at https://geosaurus.maps.arcgis.com/sharing/rest/>,
 <ArcGIS Insights License at https://geosaurus.maps.arcgis.com/sharing/rest/>,
 <Esri Redistricting License at https://geosaurus.maps.arcgis.com/sharing/rest/>,
 <ArcGIS Pro License at https://geosaurus.maps.arcgis.com/sharing/rest/>,
 <ArcGIS Drone2Map License at https://geosaurus.maps.arcgis.com/sharing/rest/>,
 <ArcGIS Image for ArcGIS Online License at https://geosaurus.maps.arcgis.com/sharing/rest/>,
 <ArcGIS GeoBIM License at https://geosaurus.maps.arcgis.com/sharing/rest/>,
 <ArcGIS for Power BI License at https://geosaurus.maps.arcgis.com/sharing/rest/>,
 <ArcGIS GeoPlanner License at https://geosaurus.maps.arcgis.com/sharing/rest/>,
 <ArcGIS AppStudio License at https://geosaurus.maps.arcgis.com/sharing/rest/>,
 <ArcGIS Drone2Map License at https://geosaurus.maps.arcgis.com/sharing/rest/>,
 <ArcGIS Workforce License at https://geosaurus.maps.arcgis.com/sharing/rest/>,
 <ArcGIS Community Analyst Lic

### Iterate over each license and gather statistics
##### Take each statistic and convert 'entitlements' field from a list to comma-separated string and add to gathered list

In [6]:
gathered = []
for lic in licenses:
    try:
        stats = lic.all()
        for s in stats:
            s['entitlements'] = ",".join(s['entitlements'])
        gathered.extend(stats)
    except:
        ...

### Create a Pandas DataFrame for Easy Visualization of Data

In [7]:
df = pd.DataFrame(gathered)

In [8]:
df.head()

,username,lastLogin,disconnected,entitlements
0,arcgis_python,1682923389000,False,survey123
1,amani_geosaurus,-1,False,Insights
2,bmajor_geosaurus,-1,False,Insights
3,pythondemo,-1,False,Insights
4,pythondemo,-1,False,RXOforArcGIS


In [9]:
# group by the username column
groupby = df.groupby('username')

# identify rows where the 'lastLogin' column has value -1
q = df.lastLogin == -1

# Set the values of the 'lastLogin' column to None where above line was True
df.loc[q, ['lastLogin']] = None

# Convert the 'lastLogin' column to a datetime format
df.lastLogin = pd.to_datetime(df.lastLogin, unit='ms')

##### Create additional rows for each entitlement

In [10]:
exploded_df = (df
               .copy() #create copy of original
               .set_index(df.columns.drop('entitlements',1).tolist()) #create multi-level index
               .entitlements.str.split(',', expand=True) # each entitlement value put in column
               .stack() #convert new columns into rows
               .reset_index() #reset index
               .rename(columns={0:'entitlements'}) #name of column from stack operation is renamed
               .loc[:, df.columns] #filter to include only original columns
)

##### Examine and analyze each group separately based on entitlement

In [11]:
grouped_df = exploded_df.groupby('entitlements')

# iterate over each group and get key and group data
for key, item in grouped_df:
    print(f"entitlement: {key}")
    display.display(grouped_df.get_group(key))

entitlement: 3DAnalystN


,username,lastLogin,disconnected,entitlements
13,arcgis_python,2023-06-09 08:25:14,False,3DAnalystN
24,bmajor_geosaurus,NaT,False,3DAnalystN
42,jyaist_geosaurus,2023-06-02 20:35:28,False,3DAnalystN
50,NGiner_geosaurus,2023-06-09 08:34:39,False,3DAnalystN
59,ptuteja_geosaurus,NaT,False,3DAnalystN
82,pythondemo,NaT,False,3DAnalystN
112,ssong_geosaurus,2023-03-09 23:34:12,False,3DAnalystN
137,testaccountreset,NaT,False,3DAnalystN
139,wrawlings_geosaurus,2021-07-27 20:30:56,False,3DAnalystN


entitlement: BusinessAnlyst


,username,lastLogin,disconnected,entitlements
179,amani_geosaurus,2020-07-23 13:41:35,False,BusinessAnlyst
180,andrew57,2022-02-24 19:21:35,False,BusinessAnlyst
181,jmccune_geosaurus,2022-02-24 19:24:42,False,BusinessAnlyst
182,naubry_geosaurus,2022-02-24 18:49:58,False,BusinessAnlyst
183,pythondemo,NaT,False,BusinessAnlyst


entitlement: CommunityAnlyst


...

,username,lastLogin,disconnected,entitlements
23,arcgis_python,2023-06-09 08:25:14,False,spatialAnalystN
35,bmajor_geosaurus,NaT,False,spatialAnalystN
39,demo_geosaurus,2022-03-10 23:40:16,False,spatialAnalystN
47,jyaist_geosaurus,2023-06-02 20:35:28,False,spatialAnalystN
57,NGiner_geosaurus,2023-06-09 08:34:39,False,spatialAnalystN
80,ptuteja_geosaurus,NaT,False,spatialAnalystN
103,pythondemo,NaT,False,spatialAnalystN
135,ssong_geosaurus,2023-03-09 23:34:12,False,spatialAnalystN
162,wrawlings_geosaurus,2021-07-27 20:30:56,False,spatialAnalystN


entitlement: survey123


,username,lastLogin,disconnected,entitlements
0,arcgis_python,2023-05-01 06:43:09,False,survey123


entitlement: workflowMgrN


,username,lastLogin,disconnected,entitlements
36,bmajor_geosaurus,NaT,False,workflowMgrN
40,demo_geosaurus,2022-03-10 23:40:16,False,workflowMgrN
81,ptuteja_geosaurus,NaT,False,workflowMgrN
104,pythondemo,NaT,False,workflowMgrN
136,ssong_geosaurus,2023-03-09 23:34:12,False,workflowMgrN
163,wrawlings_geosaurus,2021-07-27 20:30:56,False,workflowMgrN


### Total Entitlement and Extension Count

In [12]:
exploded_df.groupby('username').username.count()

username
ArcGISPyAPIBot            1
NGiner_geosaurus          8
amani_geosaurus           4
andrew57                 11
api_data_owner            1
arcgis_python            14
bmajor_geosaurus         14
cwhitmore_geosaurus       1
demo_geosaurus            5
dyaw_geosaurus            1
jmccune_geosaurus         1
jroebuck_geosaurus        3
jyaist_geosaurus          6
naubry_geosaurus          3
nparavicini_geosaurus     1
ptuteja_geosaurus        23
pythondemo               32
rsingh_geosaurus          7
ssong_geosaurus          25
testaccountreset          2
wrawlings_geosaurus      25
Name: username, dtype: int64